In [1]:
# Instalación de Prophet y dependencias necesarias
# Prophet requiere Python 3.8-3.10 para mejor compatibilidad
# Si tienes Python 3.11+, puede requerir instalación desde fuente o usar prophet==1.1.4

import sys
import subprocess

def install_package(package):
    """Instala un paquete usando pip si no está instalado."""
    try:
        __import__(package)
        print(f"✓ {package} ya está instalado")
    except ImportError:
        print(f"Instalando {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✓ {package} instalado correctamente")

# Instalar Prophet
# Nota: Si tienes problemas con la instalación, prueba:
# pip install prophet==1.1.4
# o
# pip install prophet[all]
try:
    from prophet import Prophet
    print("✓ Prophet ya está instalado")
except ImportError:
    print("Instalando Prophet...")
    try:
        # Intentar instalar la versión estándar
        subprocess.check_call([sys.executable, "-m", "pip", "install", "prophet"])
        print("✓ Prophet instalado correctamente")
        # Verificar que se puede importar
        from prophet import Prophet
    except Exception as e:
        # Si falla, intentar versión específica compatible
        print(f"Error con instalación estándar: {e}")
        print("Intentando instalar versión compatible de Prophet...")
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "prophet==1.1.4"])
            print("✓ Prophet instalado correctamente (versión 1.1.4)")
            from prophet import Prophet
        except Exception as e2:
            print(f"Error con instalación de versión específica: {e2}")
            print("Por favor, instala Prophet manualmente ejecutando:")
            print("  pip install prophet")
            print("o")
            print("  pip install prophet==1.1.4")

# Verificar versión de Python
print(f"\nVersión de Python: {sys.version}")
print(f"Versión de Python (solo número): {sys.version_info.major}.{sys.version_info.minor}")

# Nota sobre compatibilidad
if sys.version_info >= (3, 11):
    print("\n⚠️ Advertencia: Python 3.11+ puede tener problemas de compatibilidad con Prophet.")
    print("Si encuentras errores, considera usar Python 3.8-3.10 o instalar desde fuente.")
elif sys.version_info >= (3, 8):
    print("\n✓ Versión de Python compatible con Prophet (3.8-3.10)")
else:
    print("\n⚠️ Advertencia: Python < 3.8 puede no ser compatible con Prophet.")


Importing plotly failed. Interactive plots will not work.


✓ Prophet ya está instalado

Versión de Python: 3.11.5 (main, Aug 24 2023, 15:18:16) [Clang 14.0.3 (clang-1403.0.22.14.1)]
Versión de Python (solo número): 3.11

⚠️ Advertencia: Python 3.11+ puede tener problemas de compatibilidad con Prophet.
Si encuentras errores, considera usar Python 3.8-3.10 o instalar desde fuente.


# Modelo Prophet

Este notebook implementa modelos Prophet para predecir las ventas de dos productos, utilizando tres tipos de validación temporal:
- Walk-Forward Validation
- Rolling Window
- Expanding Window

**Características del modelo:**
- Prophet es un modelo de pronóstico de series temporales desarrollado por Facebook
- Maneja automáticamente tendencias, estacionalidad y efectos de días festivos
- Utiliza Optimización Bayesiana para encontrar los mejores hiperparámetros
- Entrena el modelo final para producción con el mejor método de validación


In [2]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from prophet import Prophet
import warnings
import optuna
warnings.filterwarnings('ignore')


In [3]:
# Cargar el dataset
df = pd.read_csv('data-set.csv', index_col=0)
print("Dataset cargado:")
print(f"Forma del dataset: {df.shape}")
print(f"\nPrimeras filas:")
print(df.head())
print(f"\nÚltimas filas:")
print(df.tail())
print(f"\nInformación del dataset:")
print(df.info())


Dataset cargado:
Forma del dataset: (127, 2)

Primeras filas:
    producto1   producto2
1  500.000000  200.000000
2  497.400893  210.686220
3  478.605317  222.018584
4  486.454125  233.920990
5  479.695678  238.402098

Últimas filas:
      producto1   producto2
123  164.610771  629.293034
124  150.881839  637.099467
125  151.788470  653.155282
126  137.047639  672.528345
127  141.990873  676.058092

Información del dataset:
<class 'pandas.core.frame.DataFrame'>
Index: 127 entries, 1 to 127
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   producto1  127 non-null    float64
 1   producto2  127 non-null    float64
dtypes: float64(2)
memory usage: 3.0 KB
None


## Función Prophet

Implementación del modelo Prophet que permite especificar hiperparámetros:
- **changepoint_prior_scale**: Controla la flexibilidad de la tendencia
- **seasonality_prior_scale**: Controla la fuerza de la estacionalidad
- **seasonality_mode**: Modo de estacionalidad ('additive' o 'multiplicative')
- **yearly_seasonality**: Estacionalidad anual
- **weekly_seasonality**: Estacionalidad semanal
- **daily_seasonality**: Estacionalidad diaria


In [4]:
def prophet_forecast(data, changepoint_prior_scale=0.05, seasonality_prior_scale=10.0, 
                     seasonality_mode='additive', yearly_seasonality='auto', 
                     weekly_seasonality=False, daily_seasonality=False, forecast_horizon=1):
    """
    Calcula el modelo Prophet y realiza predicciones.
    
    Parameters:
    -----------
    data : array-like
        Serie temporal de datos
    changepoint_prior_scale : float
        Controla la flexibilidad de la tendencia (por defecto 0.05)
    seasonality_prior_scale : float
        Controla la fuerza de la estacionalidad (por defecto 10.0)
    seasonality_mode : str
        Modo de estacionalidad: 'additive' o 'multiplicative' (por defecto 'additive')
    yearly_seasonality : str or bool
        Estacionalidad anual (por defecto 'auto')
    weekly_seasonality : bool
        Estacionalidad semanal (por defecto False)
    daily_seasonality : bool
        Estacionalidad diaria (por defecto False)
    forecast_horizon : int
        Número de períodos a predecir (por defecto 1)
    
    Returns:
    --------
    float or array
        Predicción(es) usando Prophet
    """
    if len(data) < 2:
        return data[-1] if len(data) > 0 else 0
    
    # Crear DataFrame con formato requerido por Prophet (ds, y)
    # Asumimos que los datos son mensuales y creamos fechas mensuales
    n = len(data)
    start_date = pd.Timestamp('2020-01-01')  # Fecha inicial arbitraria
    dates = pd.date_range(start=start_date, periods=n, freq='M')
    
    df_prophet = pd.DataFrame({
        'ds': dates,
        'y': data
    })
    
    try:
        # Crear y ajustar el modelo Prophet
        model = Prophet(
            changepoint_prior_scale=changepoint_prior_scale,
            seasonality_prior_scale=seasonality_prior_scale,
            seasonality_mode=seasonality_mode,
            yearly_seasonality=yearly_seasonality,
            weekly_seasonality=weekly_seasonality,
            daily_seasonality=daily_seasonality
        )
        
        model.fit(df_prophet)
        
        # Crear DataFrame para predicciones futuras
        future = model.make_future_dataframe(periods=forecast_horizon, freq='M')
        forecast = model.predict(future)
        
        # Obtener las predicciones futuras (últimos forecast_horizon valores)
        predictions = forecast['yhat'].tail(forecast_horizon).values
        
        # Si solo se predice un paso, devolver un escalar
        if forecast_horizon == 1:
            return float(predictions[0])
        else:
            return predictions
        
    except Exception as e:
        # En caso de error, usar el último valor como predicción
        print(f"Advertencia en Prophet: {e}")
        return data[-1]


## Función 1: Walk-Forward Validation
    
En Walk-Forward Validation, se entrena el modelo con datos hasta un punto específico y se valida con el siguiente punto. Luego se avanza un paso y se repite el proceso.


In [5]:
def walk_forward_validation(dataset, changepoint_prior_scale=0.05, seasonality_prior_scale=10.0,
                            seasonality_mode='additive', yearly_seasonality='auto',
                            weekly_seasonality=False, daily_seasonality=False, train_size_min=24):
    """
    Realiza validación Walk-Forward para cada producto del dataset.
    
    Parameters:
    -----------
    dataset : pandas.DataFrame
        Dataset con las columnas de productos
    changepoint_prior_scale : float
        Controla la flexibilidad de la tendencia
    seasonality_prior_scale : float
        Controla la fuerza de la estacionalidad
    seasonality_mode : str
        Modo de estacionalidad: 'additive' o 'multiplicative'
    yearly_seasonality : str or bool
        Estacionalidad anual
    weekly_seasonality : bool
        Estacionalidad semanal
    daily_seasonality : bool
        Estacionalidad diaria
    train_size_min : int
        Tamaño mínimo de entrenamiento antes de comenzar la validación (por defecto 24)
    
    Returns:
    --------
    dict
        Diccionario con métricas para cada producto:
        - 'producto1': lista de diccionarios con métricas por iteración
        - 'producto2': lista de diccionarios con métricas por iteración
        Cada diccionario contiene: iteracion, ventana, rmse, mae
    """
    results = {}
    
    for producto in dataset.columns:
        print(f"\n=== Walk-Forward Validation para {producto} ===")
        data = dataset[producto].values
        n = len(data)
        
        metrics_list = []
        
        # Comenzar validación desde train_size_min hasta n-1
        for i in range(train_size_min, n):
            # Datos de entrenamiento: desde el inicio hasta i
            train_data = data[:i]
            # Dato de validación: i
            actual = data[i]
            
            # Realizar predicción
            try:
                prediction = prophet_forecast(
                    train_data,
                    changepoint_prior_scale=changepoint_prior_scale,
                    seasonality_prior_scale=seasonality_prior_scale,
                    seasonality_mode=seasonality_mode,
                    yearly_seasonality=yearly_seasonality,
                    weekly_seasonality=weekly_seasonality,
                    daily_seasonality=daily_seasonality,
                    forecast_horizon=1
                )
            except Exception as e:
                print(f"Error en iteración {i}: {e}")
                prediction = train_data[-1]  # Fallback al último valor
            
            # Calcular métricas
            rmse = np.sqrt(mean_squared_error([actual], [prediction]))
            mae = mean_absolute_error([actual], [prediction])
            
            metrics_list.append({
                'iteracion': i - train_size_min + 1,
                'ventana': i,
                'rmse': rmse,
                'mae': mae
            })
            
            if (i - train_size_min + 1) % 20 == 0:
                print(f"  Iteración {i - train_size_min + 1}/{n - train_size_min}: RMSE={rmse:.4f}, MAE={mae:.4f}")
        
        results[producto] = metrics_list
        print(f"  Total de iteraciones: {len(metrics_list)}")
    
    return results


In [6]:
def rolling_window_validation(dataset, changepoint_prior_scale=0.05, seasonality_prior_scale=10.0,
                             seasonality_mode='additive', yearly_seasonality='auto',
                             weekly_seasonality=False, daily_seasonality=False, train_window_size=24):
    """
    Realiza validación Rolling Window para cada producto del dataset.
    
    Parameters:
    -----------
    dataset : pandas.DataFrame
        Dataset con las columnas de productos
    changepoint_prior_scale : float
        Controla la flexibilidad de la tendencia
    seasonality_prior_scale : float
        Controla la fuerza de la estacionalidad
    seasonality_mode : str
        Modo de estacionalidad: 'additive' o 'multiplicative'
    yearly_seasonality : str or bool
        Estacionalidad anual
    weekly_seasonality : bool
        Estacionalidad semanal
    daily_seasonality : bool
        Estacionalidad diaria
    train_window_size : int
        Tamaño fijo de la ventana de entrenamiento (por defecto 24)
    
    Returns:
    --------
    dict
        Diccionario con métricas para cada producto:
        - 'producto1': lista de diccionarios con métricas por iteración
        - 'producto2': lista de diccionarios con métricas por iteración
        Cada diccionario contiene: iteracion, ventana, rmse, mae
    """
    results = {}
    
    for producto in dataset.columns:
        print(f"\n=== Rolling Window Validation para {producto} ===")
        data = dataset[producto].values
        n = len(data)
        
        metrics_list = []
        
        # Comenzar desde train_window_size hasta n-1
        for i in range(train_window_size, n):
            # Datos de entrenamiento: ventana fija de tamaño train_window_size
            train_data = data[i - train_window_size:i]
            # Dato de validación: i
            actual = data[i]
            
            # Realizar predicción
            try:
                prediction = prophet_forecast(
                    train_data,
                    changepoint_prior_scale=changepoint_prior_scale,
                    seasonality_prior_scale=seasonality_prior_scale,
                    seasonality_mode=seasonality_mode,
                    yearly_seasonality=yearly_seasonality,
                    weekly_seasonality=weekly_seasonality,
                    daily_seasonality=daily_seasonality,
                    forecast_horizon=1
                )
            except Exception as e:
                print(f"Error en iteración {i}: {e}")
                prediction = train_data[-1]  # Fallback al último valor
            
            # Calcular métricas
            rmse = np.sqrt(mean_squared_error([actual], [prediction]))
            mae = mean_absolute_error([actual], [prediction])
            
            metrics_list.append({
                'iteracion': i - train_window_size + 1,
                'ventana': f"{i - train_window_size}-{i}",
                'rmse': rmse,
                'mae': mae
            })
            
            if (i - train_window_size + 1) % 20 == 0:
                print(f"  Iteración {i - train_window_size + 1}/{n - train_window_size}: RMSE={rmse:.4f}, MAE={mae:.4f}")
        
        results[producto] = metrics_list
        print(f"  Total de iteraciones: {len(metrics_list)}")
    
    return results


## Función 3: Expanding Window Validation

En Expanding Window Validation, la ventana de entrenamiento crece con cada iteración, comenzando desde un tamaño mínimo y expandiéndose hasta incluir todos los datos disponibles hasta ese punto.


In [7]:
def expanding_window_validation(dataset, changepoint_prior_scale=0.05, seasonality_prior_scale=10.0,
                               seasonality_mode='additive', yearly_seasonality='auto',
                               weekly_seasonality=False, daily_seasonality=False, train_size_min=24):
    """
    Realiza validación Expanding Window para cada producto del dataset.
    
    Parameters:
    -----------
    dataset : pandas.DataFrame
        Dataset con las columnas de productos
    changepoint_prior_scale : float
        Controla la flexibilidad de la tendencia
    seasonality_prior_scale : float
        Controla la fuerza de la estacionalidad
    seasonality_mode : str
        Modo de estacionalidad: 'additive' o 'multiplicative'
    yearly_seasonality : str or bool
        Estacionalidad anual
    weekly_seasonality : bool
        Estacionalidad semanal
    daily_seasonality : bool
        Estacionalidad diaria
    train_size_min : int
        Tamaño mínimo inicial de la ventana de entrenamiento (por defecto 24)
    
    Returns:
    --------
    dict
        Diccionario con métricas para cada producto:
        - 'producto1': lista de diccionarios con métricas por iteración
        - 'producto2': lista de diccionarios con métricas por iteración
        Cada diccionario contiene: iteracion, ventana, rmse, mae
    """
    results = {}
    
    for producto in dataset.columns:
        print(f"\n=== Expanding Window Validation para {producto} ===")
        data = dataset[producto].values
        n = len(data)
        
        metrics_list = []
        
        # Comenzar validación desde train_size_min hasta n-1
        for i in range(train_size_min, n):
            # Datos de entrenamiento: desde el inicio hasta i (ventana que se expande)
            train_data = data[:i]
            # Dato de validación: i
            actual = data[i]
            
            # Realizar predicción
            try:
                prediction = prophet_forecast(
                    train_data,
                    changepoint_prior_scale=changepoint_prior_scale,
                    seasonality_prior_scale=seasonality_prior_scale,
                    seasonality_mode=seasonality_mode,
                    yearly_seasonality=yearly_seasonality,
                    weekly_seasonality=weekly_seasonality,
                    daily_seasonality=daily_seasonality,
                    forecast_horizon=1
                )
            except Exception as e:
                print(f"Error en iteración {i}: {e}")
                prediction = train_data[-1]  # Fallback al último valor
            
            # Calcular métricas
            rmse = np.sqrt(mean_squared_error([actual], [prediction]))
            mae = mean_absolute_error([actual], [prediction])
            
            metrics_list.append({
                'iteracion': i - train_size_min + 1,
                'ventana': i,  # Tamaño de la ventana que se expande
                'rmse': rmse,
                'mae': mae
            })
            
            if (i - train_size_min + 1) % 20 == 0:
                print(f"  Iteración {i - train_size_min + 1}/{n - train_size_min}: RMSE={rmse:.4f}, MAE={mae:.4f}")
        
        results[producto] = metrics_list
        print(f"  Total de iteraciones: {len(metrics_list)}")
    
    return results


## Ejecución de las Validaciones

Después de definir las funciones que permitirán ejecutar el modelo teniendo en cuenta los tres tipos de validación, ahora ejecutamos las tres validaciones para cada producto y calculamos las métricas promedio.

**Configuración inicial del modelo Prophet:**
- Parámetros por defecto para la primera evaluación


In [8]:
# Parámetros de validación
TRAIN_SIZE_MIN = 24  # Tamaño mínimo de entrenamiento
TRAIN_WINDOW_SIZE = 24  # Tamaño fijo para Rolling Window

# Parámetros iniciales de Prophet (se optimizarán después)
CHANGEPOINT_PRIOR_SCALE = 0.05
SEASONALITY_PRIOR_SCALE = 10.0
SEASONALITY_MODE = 'additive'
YEARLY_SEASONALITY = 'auto'
WEEKLY_SEASONALITY = False
DAILY_SEASONALITY = False

print("=" * 60)
print("CONFIGURACIÓN INICIAL DEL MODELO PROPHET")
print("=" * 60)
print(f"Tamaño mínimo de entrenamiento: {TRAIN_SIZE_MIN}")
print(f"Tamaño de ventana fija (Rolling): {TRAIN_WINDOW_SIZE}")
print(f"Changepoint Prior Scale: {CHANGEPOINT_PRIOR_SCALE}")
print(f"Seasonality Prior Scale: {SEASONALITY_PRIOR_SCALE}")
print(f"Seasonality Mode: {SEASONALITY_MODE}")


CONFIGURACIÓN INICIAL DEL MODELO PROPHET
Tamaño mínimo de entrenamiento: 24
Tamaño de ventana fija (Rolling): 24
Changepoint Prior Scale: 0.05
Seasonality Prior Scale: 10.0
Seasonality Mode: additive


In [9]:
# Ejecutar Walk-Forward Validation para cada producto
print("\n" + "=" * 60)
print("WALK-FORWARD VALIDATION")
print("=" * 60)

results_wf = {}

for producto in df.columns:
    print(f"\n--- {producto} ---")
    results_wf[producto] = walk_forward_validation(
        df[[producto]],
        changepoint_prior_scale=CHANGEPOINT_PRIOR_SCALE,
        seasonality_prior_scale=SEASONALITY_PRIOR_SCALE,
        seasonality_mode=SEASONALITY_MODE,
        yearly_seasonality=YEARLY_SEASONALITY,
        weekly_seasonality=WEEKLY_SEASONALITY,
        daily_seasonality=DAILY_SEASONALITY,
        train_size_min=TRAIN_SIZE_MIN
    )[producto]


21:17:10 - cmdstanpy - INFO - Chain [1] start processing



WALK-FORWARD VALIDATION

--- producto1 ---

=== Walk-Forward Validation para producto1 ===


21:17:10 - cmdstanpy - INFO - Chain [1] done processing
21:17:11 - cmdstanpy - INFO - Chain [1] start processing
21:17:23 - cmdstanpy - INFO - Chain [1] done processing
21:17:23 - cmdstanpy - INFO - Chain [1] start processing
21:17:23 - cmdstanpy - INFO - Chain [1] done processing
21:17:23 - cmdstanpy - INFO - Chain [1] start processing
21:17:23 - cmdstanpy - INFO - Chain [1] done processing
21:17:23 - cmdstanpy - INFO - Chain [1] start processing
21:17:23 - cmdstanpy - INFO - Chain [1] done processing
21:17:24 - cmdstanpy - INFO - Chain [1] start processing
21:17:24 - cmdstanpy - INFO - Chain [1] done processing
21:17:24 - cmdstanpy - INFO - Chain [1] start processing
21:17:24 - cmdstanpy - INFO - Chain [1] done processing
21:17:24 - cmdstanpy - INFO - Chain [1] start processing
21:17:24 - cmdstanpy - INFO - Chain [1] done processing
21:17:24 - cmdstanpy - INFO - Chain [1] start processing
21:17:24 - cmdstanpy - INFO - Chain [1] done processing
21:17:24 - cmdstanpy - INFO - Chain [1] 

  Iteración 20/103: RMSE=6.9300, MAE=6.9300


21:17:29 - cmdstanpy - INFO - Chain [1] done processing
21:17:29 - cmdstanpy - INFO - Chain [1] start processing
21:17:29 - cmdstanpy - INFO - Chain [1] done processing
21:17:29 - cmdstanpy - INFO - Chain [1] start processing
21:17:30 - cmdstanpy - INFO - Chain [1] done processing
21:17:30 - cmdstanpy - INFO - Chain [1] start processing
21:17:30 - cmdstanpy - INFO - Chain [1] done processing
21:17:30 - cmdstanpy - INFO - Chain [1] start processing
21:17:31 - cmdstanpy - INFO - Chain [1] done processing
21:17:31 - cmdstanpy - INFO - Chain [1] start processing
21:17:31 - cmdstanpy - INFO - Chain [1] done processing
21:17:31 - cmdstanpy - INFO - Chain [1] start processing
21:17:32 - cmdstanpy - INFO - Chain [1] done processing
21:17:32 - cmdstanpy - INFO - Chain [1] start processing
21:17:32 - cmdstanpy - INFO - Chain [1] done processing
21:17:32 - cmdstanpy - INFO - Chain [1] start processing
21:17:33 - cmdstanpy - INFO - Chain [1] done processing
21:17:33 - cmdstanpy - INFO - Chain [1] 

...

  Iteración 80/103: RMSE=12.3397, MAE=12.3397


21:19:11 - cmdstanpy - INFO - Chain [1] start processing
21:19:11 - cmdstanpy - INFO - Chain [1] done processing
21:19:11 - cmdstanpy - INFO - Chain [1] start processing
21:19:11 - cmdstanpy - INFO - Chain [1] done processing
21:19:11 - cmdstanpy - INFO - Chain [1] start processing
21:19:11 - cmdstanpy - INFO - Chain [1] done processing
21:19:11 - cmdstanpy - INFO - Chain [1] start processing
21:19:11 - cmdstanpy - INFO - Chain [1] done processing
21:19:11 - cmdstanpy - INFO - Chain [1] start processing
21:19:11 - cmdstanpy - INFO - Chain [1] done processing
21:19:11 - cmdstanpy - INFO - Chain [1] start processing
21:19:11 - cmdstanpy - INFO - Chain [1] done processing
21:19:12 - cmdstanpy - INFO - Chain [1] start processing
21:19:12 - cmdstanpy - INFO - Chain [1] done processing
21:19:12 - cmdstanpy - INFO - Chain [1] start processing
21:19:12 - cmdstanpy - INFO - Chain [1] done processing
21:19:12 - cmdstanpy - INFO - Chain [1] start processing
21:19:12 - cmdstanpy - INFO - Chain [1]

  Iteración 100/103: RMSE=32.1363, MAE=32.1363


21:19:13 - cmdstanpy - INFO - Chain [1] start processing
21:19:13 - cmdstanpy - INFO - Chain [1] done processing


  Total de iteraciones: 103


In [10]:
# Ejecutar Rolling Window Validation para cada producto
print("\n" + "=" * 60)
print("ROLLING WINDOW VALIDATION")
print("=" * 60)

results_rw = {}

for producto in df.columns:
    print(f"\n--- {producto} ---")
    results_rw[producto] = rolling_window_validation(
        df[[producto]],
        changepoint_prior_scale=CHANGEPOINT_PRIOR_SCALE,
        seasonality_prior_scale=SEASONALITY_PRIOR_SCALE,
        seasonality_mode=SEASONALITY_MODE,
        yearly_seasonality=YEARLY_SEASONALITY,
        weekly_seasonality=WEEKLY_SEASONALITY,
        daily_seasonality=DAILY_SEASONALITY,
        train_window_size=TRAIN_WINDOW_SIZE
    )[producto]


21:19:14 - cmdstanpy - INFO - Chain [1] start processing
21:19:14 - cmdstanpy - INFO - Chain [1] done processing
21:19:14 - cmdstanpy - INFO - Chain [1] start processing



ROLLING WINDOW VALIDATION

--- producto1 ---

=== Rolling Window Validation para producto1 ===


21:19:14 - cmdstanpy - INFO - Chain [1] done processing
21:19:14 - cmdstanpy - INFO - Chain [1] start processing
21:19:14 - cmdstanpy - INFO - Chain [1] done processing
21:19:14 - cmdstanpy - INFO - Chain [1] start processing
21:19:14 - cmdstanpy - INFO - Chain [1] done processing
21:19:14 - cmdstanpy - INFO - Chain [1] start processing
21:19:14 - cmdstanpy - INFO - Chain [1] done processing
21:19:14 - cmdstanpy - INFO - Chain [1] start processing
21:19:14 - cmdstanpy - INFO - Chain [1] done processing
21:19:14 - cmdstanpy - INFO - Chain [1] start processing
21:19:14 - cmdstanpy - INFO - Chain [1] done processing
21:19:14 - cmdstanpy - INFO - Chain [1] start processing
21:19:15 - cmdstanpy - INFO - Chain [1] done processing
21:19:15 - cmdstanpy - INFO - Chain [1] start processing
21:19:15 - cmdstanpy - INFO - Chain [1] done processing
21:19:15 - cmdstanpy - INFO - Chain [1] start processing
21:19:15 - cmdstanpy - INFO - Chain [1] done processing
21:19:15 - cmdstanpy - INFO - Chain [1] 

  Iteración 20/103: RMSE=3.0519, MAE=3.0519


21:19:17 - cmdstanpy - INFO - Chain [1] done processing
21:19:17 - cmdstanpy - INFO - Chain [1] start processing
21:19:17 - cmdstanpy - INFO - Chain [1] done processing
21:19:17 - cmdstanpy - INFO - Chain [1] start processing
21:19:17 - cmdstanpy - INFO - Chain [1] done processing
21:19:17 - cmdstanpy - INFO - Chain [1] start processing
21:19:17 - cmdstanpy - INFO - Chain [1] done processing
21:19:17 - cmdstanpy - INFO - Chain [1] start processing
21:19:17 - cmdstanpy - INFO - Chain [1] done processing
21:19:17 - cmdstanpy - INFO - Chain [1] start processing
21:19:17 - cmdstanpy - INFO - Chain [1] done processing
21:19:18 - cmdstanpy - INFO - Chain [1] start processing
21:19:18 - cmdstanpy - INFO - Chain [1] done processing
21:19:18 - cmdstanpy - INFO - Chain [1] start processing
21:19:18 - cmdstanpy - INFO - Chain [1] done processing
21:19:18 - cmdstanpy - INFO - Chain [1] start processing
21:19:18 - cmdstanpy - INFO - Chain [1] done processing
21:19:18 - cmdstanpy - INFO - Chain [1] 

...

  Iteración 80/103: RMSE=9.8092, MAE=9.8092


21:19:40 - cmdstanpy - INFO - Chain [1] done processing
21:19:40 - cmdstanpy - INFO - Chain [1] start processing
21:19:40 - cmdstanpy - INFO - Chain [1] done processing
21:19:40 - cmdstanpy - INFO - Chain [1] start processing
21:19:40 - cmdstanpy - INFO - Chain [1] done processing
21:19:40 - cmdstanpy - INFO - Chain [1] start processing
21:19:40 - cmdstanpy - INFO - Chain [1] done processing
21:19:41 - cmdstanpy - INFO - Chain [1] start processing
21:19:41 - cmdstanpy - INFO - Chain [1] done processing
21:19:41 - cmdstanpy - INFO - Chain [1] start processing
21:19:41 - cmdstanpy - INFO - Chain [1] done processing
21:19:41 - cmdstanpy - INFO - Chain [1] start processing
21:19:41 - cmdstanpy - INFO - Chain [1] done processing
21:19:41 - cmdstanpy - INFO - Chain [1] start processing
21:19:41 - cmdstanpy - INFO - Chain [1] done processing
21:19:41 - cmdstanpy - INFO - Chain [1] start processing
21:19:41 - cmdstanpy - INFO - Chain [1] done processing
21:19:41 - cmdstanpy - INFO - Chain [1] 

  Iteración 100/103: RMSE=4.4023, MAE=4.4023


21:19:43 - cmdstanpy - INFO - Chain [1] done processing
21:19:43 - cmdstanpy - INFO - Chain [1] start processing
21:19:44 - cmdstanpy - INFO - Chain [1] done processing


  Total de iteraciones: 103


In [11]:
# Ejecutar Expanding Window Validation para cada producto
print("\n" + "=" * 60)
print("EXPANDING WINDOW VALIDATION")
print("=" * 60)

results_ew = {}

for producto in df.columns:
    print(f"\n--- {producto} ---")
    results_ew[producto] = expanding_window_validation(
        df[[producto]],
        changepoint_prior_scale=CHANGEPOINT_PRIOR_SCALE,
        seasonality_prior_scale=SEASONALITY_PRIOR_SCALE,
        seasonality_mode=SEASONALITY_MODE,
        yearly_seasonality=YEARLY_SEASONALITY,
        weekly_seasonality=WEEKLY_SEASONALITY,
        daily_seasonality=DAILY_SEASONALITY,
        train_size_min=TRAIN_SIZE_MIN
    )[producto]


21:19:44 - cmdstanpy - INFO - Chain [1] start processing
21:19:44 - cmdstanpy - INFO - Chain [1] done processing
21:19:44 - cmdstanpy - INFO - Chain [1] start processing



EXPANDING WINDOW VALIDATION

--- producto1 ---

=== Expanding Window Validation para producto1 ===


21:19:55 - cmdstanpy - INFO - Chain [1] done processing
21:19:55 - cmdstanpy - INFO - Chain [1] start processing
21:19:55 - cmdstanpy - INFO - Chain [1] done processing
21:19:55 - cmdstanpy - INFO - Chain [1] start processing
21:19:55 - cmdstanpy - INFO - Chain [1] done processing
21:19:55 - cmdstanpy - INFO - Chain [1] start processing
21:19:56 - cmdstanpy - INFO - Chain [1] done processing
21:19:56 - cmdstanpy - INFO - Chain [1] start processing
21:19:56 - cmdstanpy - INFO - Chain [1] done processing
21:19:56 - cmdstanpy - INFO - Chain [1] start processing
21:19:56 - cmdstanpy - INFO - Chain [1] done processing
21:19:56 - cmdstanpy - INFO - Chain [1] start processing
21:19:56 - cmdstanpy - INFO - Chain [1] done processing
21:19:57 - cmdstanpy - INFO - Chain [1] start processing
21:19:57 - cmdstanpy - INFO - Chain [1] done processing
21:19:57 - cmdstanpy - INFO - Chain [1] start processing
21:19:57 - cmdstanpy - INFO - Chain [1] done processing
21:19:57 - cmdstanpy - INFO - Chain [1] 

  Iteración 20/103: RMSE=6.9300, MAE=6.9300


21:20:01 - cmdstanpy - INFO - Chain [1] done processing
21:20:01 - cmdstanpy - INFO - Chain [1] start processing
21:20:01 - cmdstanpy - INFO - Chain [1] done processing
21:20:02 - cmdstanpy - INFO - Chain [1] start processing
21:20:02 - cmdstanpy - INFO - Chain [1] done processing
21:20:02 - cmdstanpy - INFO - Chain [1] start processing
21:20:02 - cmdstanpy - INFO - Chain [1] done processing
21:20:02 - cmdstanpy - INFO - Chain [1] start processing
21:20:03 - cmdstanpy - INFO - Chain [1] done processing
21:20:03 - cmdstanpy - INFO - Chain [1] start processing
21:20:03 - cmdstanpy - INFO - Chain [1] done processing
21:20:03 - cmdstanpy - INFO - Chain [1] start processing
21:20:04 - cmdstanpy - INFO - Chain [1] done processing
21:20:04 - cmdstanpy - INFO - Chain [1] start processing
21:20:04 - cmdstanpy - INFO - Chain [1] done processing
21:20:04 - cmdstanpy - INFO - Chain [1] start processing
21:20:05 - cmdstanpy - INFO - Chain [1] done processing
21:20:05 - cmdstanpy - INFO - Chain [1] 

...

  Iteración 80/103: RMSE=12.3397, MAE=12.3397


21:21:37 - cmdstanpy - INFO - Chain [1] start processing
21:21:37 - cmdstanpy - INFO - Chain [1] done processing
21:21:38 - cmdstanpy - INFO - Chain [1] start processing
21:21:38 - cmdstanpy - INFO - Chain [1] done processing
21:21:38 - cmdstanpy - INFO - Chain [1] start processing
21:21:38 - cmdstanpy - INFO - Chain [1] done processing
21:21:38 - cmdstanpy - INFO - Chain [1] start processing
21:21:38 - cmdstanpy - INFO - Chain [1] done processing
21:21:38 - cmdstanpy - INFO - Chain [1] start processing
21:21:38 - cmdstanpy - INFO - Chain [1] done processing
21:21:38 - cmdstanpy - INFO - Chain [1] start processing
21:21:38 - cmdstanpy - INFO - Chain [1] done processing
21:21:38 - cmdstanpy - INFO - Chain [1] start processing
21:21:38 - cmdstanpy - INFO - Chain [1] done processing
21:21:38 - cmdstanpy - INFO - Chain [1] start processing
21:21:38 - cmdstanpy - INFO - Chain [1] done processing
21:21:38 - cmdstanpy - INFO - Chain [1] start processing
21:21:38 - cmdstanpy - INFO - Chain [1]

  Iteración 100/103: RMSE=32.1363, MAE=32.1363


21:21:40 - cmdstanpy - INFO - Chain [1] start processing
21:21:40 - cmdstanpy - INFO - Chain [1] done processing


  Total de iteraciones: 103


In [12]:
def calcular_metricas_promedio(results, nombre_validacion):
    """
    Calcula las métricas promedio para cada producto.
    
    Parameters:
    -----------
    results : dict
        Resultados de la validación
    nombre_validacion : str
        Nombre del tipo de validación
    
    Returns:
    --------
    dict
        Diccionario con métricas promedio por producto
    """
    metricas_promedio = {}
    
    print(f"\n{'='*60}")
    print(f"MÉTRICAS PROMEDIO - {nombre_validacion.upper()}")
    print(f"{'='*60}")
    
    for producto in results.keys():
        metrics_list = results[producto]
        
        # Calcular promedios
        rmse_promedio = np.mean([m['rmse'] for m in metrics_list])
        mae_promedio = np.mean([m['mae'] for m in metrics_list])
        
        metricas_promedio[producto] = {
            'rmse_promedio': rmse_promedio,
            'mae_promedio': mae_promedio,
            'num_iteraciones': len(metrics_list)
        }
        
        print(f"\n{producto}:")
        print(f"  RMSE Promedio: {rmse_promedio:.4f}")
        print(f"  MAE Promedio: {mae_promedio:.4f}")
        print(f"  Número de iteraciones: {len(metrics_list)}")
    
    return metricas_promedio

# Calcular métricas promedio para cada tipo de validación
metricas_wf = calcular_metricas_promedio(results_wf, "Walk-Forward")
metricas_rw = calcular_metricas_promedio(results_rw, "Rolling Window")
metricas_ew = calcular_metricas_promedio(results_ew, "Expanding Window")



MÉTRICAS PROMEDIO - WALK-FORWARD

producto1:
  RMSE Promedio: 39.5367
  MAE Promedio: 39.5367
  Número de iteraciones: 103

producto2:
  RMSE Promedio: 33.8046
  MAE Promedio: 33.8046
  Número de iteraciones: 103

MÉTRICAS PROMEDIO - ROLLING WINDOW

producto1:
  RMSE Promedio: 14.0719
  MAE Promedio: 14.0719
  Número de iteraciones: 103

producto2:
  RMSE Promedio: 15.6982
  MAE Promedio: 15.6982
  Número de iteraciones: 103

MÉTRICAS PROMEDIO - EXPANDING WINDOW

producto1:
  RMSE Promedio: 39.5367
  MAE Promedio: 39.5367
  Número de iteraciones: 103

producto2:
  RMSE Promedio: 33.8046
  MAE Promedio: 33.8046
  Número de iteraciones: 103


## Comparación de Resultados y Selección del Mejor Método de Validación

Comparamos los RMSE promedio de cada tipo de validación para seleccionar el mejor método.


In [13]:
# Crear DataFrame comparativo
comparacion = []

for producto in df.columns:
    comparacion.append({
        'Producto': producto,
        'Validacion': 'Walk-Forward',
        'RMSE_Promedio': metricas_wf[producto]['rmse_promedio'],
        'MAE_Promedio': metricas_wf[producto]['mae_promedio']
    })
    comparacion.append({
        'Producto': producto,
        'Validacion': 'Rolling Window',
        'RMSE_Promedio': metricas_rw[producto]['rmse_promedio'],
        'MAE_Promedio': metricas_rw[producto]['mae_promedio']
    })
    comparacion.append({
        'Producto': producto,
        'Validacion': 'Expanding Window',
        'RMSE_Promedio': metricas_ew[producto]['rmse_promedio'],
        'MAE_Promedio': metricas_ew[producto]['mae_promedio']
    })

df_comparacion = pd.DataFrame(comparacion)
print("\n" + "=" * 80)
print("COMPARACIÓN DE MÉTODOS DE VALIDACIÓN")
print("=" * 80)
print(df_comparacion.to_string(index=False))

# Identificar el mejor método para cada producto
print("\n" + "=" * 80)
print("MEJOR MÉTODO DE VALIDACIÓN POR PRODUCTO (basado en RMSE Promedio)")
print("=" * 80)
mejor_validacion = {}
for producto in df.columns:
    producto_df = df_comparacion[df_comparacion['Producto'] == producto]
    mejor = producto_df.loc[producto_df['RMSE_Promedio'].idxmin()]
    mejor_validacion[producto] = mejor['Validacion']
    print(f"\n{producto}:")
    print(f"  Mejor método: {mejor['Validacion']}")
    print(f"  RMSE Promedio: {mejor['RMSE_Promedio']:.4f}")



COMPARACIÓN DE MÉTODOS DE VALIDACIÓN
 Producto       Validacion  RMSE_Promedio  MAE_Promedio
producto1     Walk-Forward      39.536692     39.536692
producto1   Rolling Window      14.071858     14.071858
producto1 Expanding Window      39.536692     39.536692
producto2     Walk-Forward      33.804644     33.804644
producto2   Rolling Window      15.698194     15.698194
producto2 Expanding Window      33.804644     33.804644

MEJOR MÉTODO DE VALIDACIÓN POR PRODUCTO (basado en RMSE Promedio)

producto1:
  Mejor método: Rolling Window
  RMSE Promedio: 14.0719

producto2:
  Mejor método: Rolling Window
  RMSE Promedio: 15.6982


## Optimización Bayesiana para Encontrar los Mejores Parámetros Prophet

Utilizamos Optimización Bayesiana (Optuna) para optimizar los hiperparámetros del modelo Prophet.


In [14]:
def objective_walk_forward(trial, data, train_size_min=24):
    """
    Función objetivo para optimización bayesiana usando Walk-Forward Validation.
    """
    changepoint_prior_scale = trial.suggest_float('changepoint_prior_scale', 0.001, 0.5, log=True)
    seasonality_prior_scale = trial.suggest_float('seasonality_prior_scale', 0.01, 50.0, log=True)
    seasonality_mode = trial.suggest_categorical('seasonality_mode', ['additive', 'multiplicative'])
    yearly_seasonality = trial.suggest_categorical('yearly_seasonality', [True, False, 'auto'])
    
    n = len(data)
    errors = []
    
    for i in range(train_size_min, n):
        train_data = data[:i]
        actual = data[i]
        
        try:
            prediction = prophet_forecast(
                train_data,
                changepoint_prior_scale=changepoint_prior_scale,
                seasonality_prior_scale=seasonality_prior_scale,
                seasonality_mode=seasonality_mode,
                yearly_seasonality=yearly_seasonality,
                weekly_seasonality=False,
                daily_seasonality=False,
                forecast_horizon=1
            )
            error_val = np.sqrt(mean_squared_error([actual], [prediction]))
            if np.isfinite(error_val):
                errors.append(error_val)
        except:
            continue
    
    if len(errors) == 0:
        return float('inf')
    
    return np.mean(errors)

def objective_rolling_window(trial, data, train_window_size=24):
    """
    Función objetivo para optimización bayesiana usando Rolling Window Validation.
    """
    changepoint_prior_scale = trial.suggest_float('changepoint_prior_scale', 0.001, 0.5, log=True)
    seasonality_prior_scale = trial.suggest_float('seasonality_prior_scale', 0.01, 50.0, log=True)
    seasonality_mode = trial.suggest_categorical('seasonality_mode', ['additive', 'multiplicative'])
    yearly_seasonality = trial.suggest_categorical('yearly_seasonality', [True, False, 'auto'])
    
    n = len(data)
    errors = []
    
    for i in range(train_window_size, n):
        train_data = data[i - train_window_size:i]
        actual = data[i]
        
        try:
            prediction = prophet_forecast(
                train_data,
                changepoint_prior_scale=changepoint_prior_scale,
                seasonality_prior_scale=seasonality_prior_scale,
                seasonality_mode=seasonality_mode,
                yearly_seasonality=yearly_seasonality,
                weekly_seasonality=False,
                daily_seasonality=False,
                forecast_horizon=1
            )
            error_val = np.sqrt(mean_squared_error([actual], [prediction]))
            if np.isfinite(error_val):
                errors.append(error_val)
        except:
            continue
    
    if len(errors) == 0:
        return float('inf')
    
    return np.mean(errors)

def objective_expanding_window(trial, data, train_size_min=24):
    """
    Función objetivo para optimización bayesiana usando Expanding Window Validation.
    """
    changepoint_prior_scale = trial.suggest_float('changepoint_prior_scale', 0.001, 0.5, log=True)
    seasonality_prior_scale = trial.suggest_float('seasonality_prior_scale', 0.01, 50.0, log=True)
    seasonality_mode = trial.suggest_categorical('seasonality_mode', ['additive', 'multiplicative'])
    yearly_seasonality = trial.suggest_categorical('yearly_seasonality', [True, False, 'auto'])
    
    n = len(data)
    errors = []
    
    for i in range(train_size_min, n):
        train_data = data[:i]
        actual = data[i]
        
        try:
            prediction = prophet_forecast(
                train_data,
                changepoint_prior_scale=changepoint_prior_scale,
                seasonality_prior_scale=seasonality_prior_scale,
                seasonality_mode=seasonality_mode,
                yearly_seasonality=yearly_seasonality,
                weekly_seasonality=False,
                daily_seasonality=False,
                forecast_horizon=1
            )
            error_val = np.sqrt(mean_squared_error([actual], [prediction]))
            if np.isfinite(error_val):
                errors.append(error_val)
        except:
            continue
    
    if len(errors) == 0:
        return float('inf')
    
    return np.mean(errors)


In [15]:
# Realizar optimización bayesiana para cada producto usando su mejor método de validación
print("\n" + "=" * 80)
print("OPTIMIZACIÓN BAYESIANA")
print("=" * 80)

mejores_parametros = {}

for producto in df.columns:
    print(f"\n--- Optimizando {producto} ---")
    data = df[producto].values
    mejor_metodo = mejor_validacion[producto]
    
    print(f"{producto}: Mejor método = {mejor_metodo}")
    
    # Seleccionar función objetivo según el mejor método
    if mejor_metodo == 'Walk-Forward':
        objective_func = lambda trial: objective_walk_forward(trial, data, TRAIN_SIZE_MIN)
    elif mejor_metodo == 'Rolling Window':
        objective_func = lambda trial: objective_rolling_window(trial, data, TRAIN_WINDOW_SIZE)
    else:  # Expanding Window
        objective_func = lambda trial: objective_expanding_window(trial, data, TRAIN_SIZE_MIN)
    
    # Crear estudio de Optuna
    study = optuna.create_study(direction='minimize')
    study.optimize(objective_func, n_trials=30, show_progress_bar=True)
    
    # Obtener mejores parámetros
    best_params = study.best_params
    best_rmse = study.best_value
    
    mejores_parametros[producto] = {
        'params': best_params,
        'rmse': best_rmse,
        'metodo': mejor_metodo
    }
    
    print(f"  Mejores parámetros encontrados:")
    for key, value in best_params.items():
        print(f"    {key}: {value}")
    print(f"  Mejor RMSE: {best_rmse:.4f}")
    print(f"  Método usado: {mejor_metodo}")


[I 2025-11-30 21:21:40,614] A new study created in memory with name: no-name-9437c3bd-a7d3-4946-9e93-d4386e08ea5f



OPTIMIZACIÓN BAYESIANA

--- Optimizando producto1 ---
producto1: Mejor método = Rolling Window


  0%|          | 0/30 [00:00<?, ?it/s]

21:21:40 - cmdstanpy - INFO - Chain [1] start processing
21:21:40 - cmdstanpy - INFO - Chain [1] done processing
21:21:40 - cmdstanpy - INFO - Chain [1] start processing
21:21:40 - cmdstanpy - INFO - Chain [1] done processing
21:21:41 - cmdstanpy - INFO - Chain [1] start processing
21:21:41 - cmdstanpy - INFO - Chain [1] done processing
21:21:41 - cmdstanpy - INFO - Chain [1] start processing
21:21:41 - cmdstanpy - INFO - Chain [1] done processing
21:21:41 - cmdstanpy - INFO - Chain [1] start processing
21:21:41 - cmdstanpy - INFO - Chain [1] done processing
21:21:41 - cmdstanpy - INFO - Chain [1] start processing
21:21:41 - cmdstanpy - INFO - Chain [1] done processing
21:21:41 - cmdstanpy - INFO - Chain [1] start processing
21:21:41 - cmdstanpy - INFO - Chain [1] done processing
21:21:41 - cmdstanpy - INFO - Chain [1] start processing
21:21:41 - cmdstanpy - INFO - Chain [1] done processing
21:21:41 - cmdstanpy - INFO - Chain [1] start processing
21:21:42 - cmdstanpy - INFO - Chain [1]

[I 2025-11-30 21:21:58,962] Trial 0 finished with value: 8.104429210502232 and parameters: {'changepoint_prior_scale': 0.08984870879298815, 'seasonality_prior_scale': 0.06613012298597096, 'seasonality_mode': 'additive', 'yearly_seasonality': False}. Best is trial 0 with value: 8.104429210502232.


...

[I 2025-11-30 23:20:07,339] Trial 27 finished with value: 14.143643975103538 and parameters: {'changepoint_prior_scale': 0.09667794871913248, 'seasonality_prior_scale': 0.09090693322406596, 'seasonality_mode': 'additive', 'yearly_seasonality': False}. Best is trial 21 with value: 12.792526273126981.


23:20:22 - cmdstanpy - INFO - Chain [1] done processing
23:20:22 - cmdstanpy - INFO - Chain [1] start processing
23:20:37 - cmdstanpy - INFO - Chain [1] done processing
23:20:37 - cmdstanpy - INFO - Chain [1] start processing
23:20:51 - cmdstanpy - INFO - Chain [1] done processing
23:20:51 - cmdstanpy - INFO - Chain [1] start processing
23:21:05 - cmdstanpy - INFO - Chain [1] done processing
23:21:05 - cmdstanpy - INFO - Chain [1] start processing
23:21:19 - cmdstanpy - INFO - Chain [1] done processing
23:21:20 - cmdstanpy - INFO - Chain [1] start processing
23:21:34 - cmdstanpy - INFO - Chain [1] done processing
23:21:34 - cmdstanpy - INFO - Chain [1] start processing
23:21:48 - cmdstanpy - INFO - Chain [1] done processing
23:21:48 - cmdstanpy - INFO - Chain [1] start processing
23:22:02 - cmdstanpy - INFO - Chain [1] done processing
23:22:02 - cmdstanpy - INFO - Chain [1] start processing
23:22:16 - cmdstanpy - INFO - Chain [1] done processing
23:22:16 - cmdstanpy - INFO - Chain [1] 

[I 2025-11-30 23:43:52,628] Trial 28 finished with value: 20.543326284319193 and parameters: {'changepoint_prior_scale': 0.25081641881646055, 'seasonality_prior_scale': 1.1782462776602993, 'seasonality_mode': 'multiplicative', 'yearly_seasonality': True}. Best is trial 21 with value: 12.792526273126981.


23:43:52 - cmdstanpy - INFO - Chain [1] start processing
23:43:52 - cmdstanpy - INFO - Chain [1] done processing
23:43:53 - cmdstanpy - INFO - Chain [1] start processing
23:43:53 - cmdstanpy - INFO - Chain [1] done processing
23:43:53 - cmdstanpy - INFO - Chain [1] start processing
23:43:53 - cmdstanpy - INFO - Chain [1] done processing
23:43:53 - cmdstanpy - INFO - Chain [1] start processing
23:43:53 - cmdstanpy - INFO - Chain [1] done processing
23:43:53 - cmdstanpy - INFO - Chain [1] start processing
23:43:53 - cmdstanpy - INFO - Chain [1] done processing
23:43:53 - cmdstanpy - INFO - Chain [1] start processing
23:43:53 - cmdstanpy - INFO - Chain [1] done processing
23:43:53 - cmdstanpy - INFO - Chain [1] start processing
23:43:54 - cmdstanpy - INFO - Chain [1] done processing
23:43:54 - cmdstanpy - INFO - Chain [1] start processing
23:43:54 - cmdstanpy - INFO - Chain [1] done processing
23:43:54 - cmdstanpy - INFO - Chain [1] start processing
23:43:54 - cmdstanpy - INFO - Chain [1]

[I 2025-11-30 23:44:11,288] Trial 29 finished with value: 29.341365287293083 and parameters: {'changepoint_prior_scale': 0.010266097322585824, 'seasonality_prior_scale': 0.9432454530039108, 'seasonality_mode': 'multiplicative', 'yearly_seasonality': 'auto'}. Best is trial 21 with value: 12.792526273126981.
  Mejores parámetros encontrados:
    changepoint_prior_scale: 0.4977207144984529
    seasonality_prior_scale: 0.5309927356445465
    seasonality_mode: multiplicative
    yearly_seasonality: False
  Mejor RMSE: 12.7925
  Método usado: Rolling Window


## Modelo Final para Producción

Entrenamos el modelo final con todos los datos disponibles usando los mejores parámetros encontrados mediante optimización bayesiana y realizamos la predicción para el siguiente mes (1 pronóstico).


In [16]:
# Entrenar modelo final y realizar predicción para producción
print("\n" + "=" * 80)
print("PREDICCIÓN FINAL PARA PRODUCCIÓN")
print("=" * 80)

predicciones_produccion = {}

for producto in df.columns:
    print(f"\n--- {producto} ---")
    data = df[producto].values
    
    # Obtener mejores parámetros de la optimización bayesiana
    params = mejores_parametros[producto]
    best_params = params['params']
    
    print(f"Parámetros del modelo:")
    for key, value in best_params.items():
        print(f"  {key}: {value}")
    print(f"  RMSE en validación: {params['rmse']:.4f}")
    print(f"  Método de validación usado: {params['metodo']}")
    
    # Entrenar modelo final con todos los datos
    try:
        prediccion = prophet_forecast(
            data,
            changepoint_prior_scale=best_params['changepoint_prior_scale'],
            seasonality_prior_scale=best_params['seasonality_prior_scale'],
            seasonality_mode=best_params['seasonality_mode'],
            yearly_seasonality=best_params['yearly_seasonality'],
            weekly_seasonality=False,
            daily_seasonality=False,
            forecast_horizon=1
        )
        
        predicciones_produccion[producto] = prediccion
        
        print(f"  Predicción para el siguiente mes: {prediccion:.4f}")
        print(f"  Último valor observado: {data[-1]:.4f}")
        print(f"  Diferencia: {prediccion - data[-1]:.4f}")
        
    except Exception as e:
        print(f"  Error al generar predicción: {e}")
        # Fallback: usar último valor
        predicciones_produccion[producto] = data[-1]
        print(f"  Predicción (fallback): {data[-1]:.4f}")

# Mostrar resumen de predicciones
print("\n" + "=" * 80)
print("RESUMEN DE PREDICCIONES PARA PRODUCCIÓN")
print("=" * 80)
for producto, prediccion in predicciones_produccion.items():
    print(f"{producto}: {prediccion:.4f}")


23:44:11 - cmdstanpy - INFO - Chain [1] start processing



PREDICCIÓN FINAL PARA PRODUCCIÓN

--- producto1 ---
Parámetros del modelo:
  changepoint_prior_scale: 0.49144377411394097
  seasonality_prior_scale: 0.01078535112134089
  seasonality_mode: additive
  yearly_seasonality: auto
  RMSE en validación: 7.2094
  Método de validación usado: Rolling Window


23:44:11 - cmdstanpy - INFO - Chain [1] done processing
23:44:11 - cmdstanpy - INFO - Chain [1] start processing
23:44:11 - cmdstanpy - INFO - Chain [1] done processing


  Predicción para el siguiente mes: 111.6571
  Último valor observado: 141.9909
  Diferencia: -30.3338

--- producto2 ---
Parámetros del modelo:
  changepoint_prior_scale: 0.4977207144984529
  seasonality_prior_scale: 0.5309927356445465
  seasonality_mode: multiplicative
  yearly_seasonality: False
  RMSE en validación: 12.7925
  Método de validación usado: Rolling Window
  Predicción para el siguiente mes: 583.1367
  Último valor observado: 676.0581
  Diferencia: -92.9214

RESUMEN DE PREDICCIONES PARA PRODUCCIÓN
producto1: 111.6571
producto2: 583.1367
